# Stability calculations

## -- LUVOIR --

In notebook 10, we calculated the PASTIS modes for all three currently designed apodizers for the LUVOIR A pupil. In this notebook, we will load the modes and calculate the maximum mode contributions $\sigma$ and strability requirements $\Delta \sigma$.

After that I also calculate the maximum segment contributions $\mu$.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging_new import LuvoirAPLC

## Instantiate a simple SM for mode plotting

I need to simplify this... I can make a child class like I did for the full LUVOIR simulator.

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(hc.UnstructuredCoords(poslist))

## Mode and eigenvalue units

Since the matrices got generated normalized to an aberration of 1 nm, everything stemming from the matrix will be in units of **nanometers**. --> This is the desired behaviour of course and is yet to be confirmed, which is one of the goals of this notebook.

In [ ]:
eunit = 1e-9

## Also instantiate a LUVOR telescope

For cumulative contrast calculation.

> **ATTENTION:** the variable `apodizer_design` has to be correct for the data we are reading with the variable `savedpath` further below!!!

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'large'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
luvoir.make_segment_zernike_primary(1)

In [ ]:
zernike_coeffs = np.zeros([luvoir.sm.num_actuators])
luvoir.sm.actuators = zernike_coeffs

In [ ]:
luvoir.sm(luvoir.wf_aper)

In [ ]:
# Make reference image
# luvoir.flatten()
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(psf_unaber/norm, norm=LogNorm())

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(ref/norm, norm=LogNorm())

In [ ]:
apod_prop = hc.Apodizer(luvoir.apodizer)
wf_sm = wf_aper
wf_apod = apod_prop(wf_sm)
plt.figure(figsize=(10, 10))
hc.imshow_field(wf_apod.amplitude)
samp_foc = luvoir.apod_dict[apodizer_design]['fpm_px'] / (luvoir.apod_dict[apodizer_design]['fpm_rad'] * 2)
focal_grid_fpm = hc.make_focal_grid(
            samp_foc,
            luvoir.apod_dict[apodizer_design]['fpm_rad'],
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )
luvoir.fpm = 1 - hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['fpm_rad']*luvoir.wvln/luvoir.diam)(focal_grid_fpm)
plt.figure(figsize=(10, 10))
hc.imshow_field(luvoir.fpm)
luvoir.coro = hc.LyotCoronagraph(luvoir.pupil_grid, luvoir.fpm, luvoir.ls)
wf_lyot = luvoir.coro(wf_apod)
plt.figure(figsize=(10, 10))
hc.imshow_field(wf_lyot.amplitude)
plt.figure(figsize=(10, 10))
hc.imshow_field(wf_lyot.phase)
focal_det = luvoir.focal_det
# focal_det = hc.make_focal_grid(
#             4,
#             60,
#             pupil_diameter=luvoir.diam,
#             focal_length=1,
#             reference_wavelength=luvoir.wvln,
#         )
luvoir.prop = hc.FraunhoferPropagator(pupil_grid,focal_det)
wf_im_coro = luvoir.prop(wf_lyot)
plt.figure(figsize=(10, 10))
hc.imshow_field(wf_im_coro.intensity, norm=LogNorm())
IWA = 13
OWA = 48
IWAmask = hc.circular_aperture(2*IWA*luvoir.wvln/luvoir.diam)(focal_det)
OWAmask = hc.circular_aperture(2*OWA*luvoir.wvln/luvoir.diam)(focal_det)
final_mask =  -IWAmask + OWAmask
plt.figure(figsize=(10, 10))
#hc.imshow_field(final_mask)
hc.imshow_field(final_mask*wf_im_coro.intensity, norm=LogNorm())
plt.figure(figsize=(10, 10))
hc.imshow_field(final_mask*np.log(wf_im_coro.intensity))
# ref = luvoir.prop(wf_apod).intensity
# norm = ref.max()
np.sum(final_mask*wf_im_coro.intensity/norm)/np.sum(final_mask)

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(psf_unaber-wf_im_coro.intensity)